## 1. 直方图的基本原理

直方图（Histogram）是一种用于展示**数值型变量分布情况**的图形工具。其原理是将数据划分为若干**连续、不重叠的区间**（称为 “bin” 或 “箱子”），统计每个区间内数据点的数量，并以矩形的高度表示频数或频率。

设有一组日收益率数据 $\{r_1, r_2, \ldots, r_n\}$，我们将其划分为 $K$ 个等宽的区间，每个区间的宽度为：

$$
h = \frac{\max(r) - \min(r)}{K}
$$

第 $k$ 个区间为 $[a_k, a_{k+1})$，其频数记为 $f_k$，那么对应的矩形高度就是 $f_k$（或标准化后的频率）。绘图过程中，横轴表示收益率区间，纵轴表示该区间的频数或频率。

## 2. 核密度函数估计（KDE）

核密度估计 (Kernel Density Estimation, KDE) 是一种用于估计未知概率密度函数的非参数方法，适用于连续型数据且不依赖于事先指定的分布形式。其基本思想是：在密度函数的每一个估计点上，根据样本点到该点的距离，使用核函数分配权重并加权平均，从而构建平滑的密度曲线。

设样本为 $x_1, x_2, \dots, x_n$，其密度函数在任意点 $x$ 上的估计形式为：

$$
\hat{f}_h(x) = \frac{1}{n h} \sum_{i=1}^{n} K\left( \frac{x - x_i}{h} \right)
$$

其中：

- $K(\cdot)$ 是核函数（kernel function），通常是一个对称的概率密度函数；
- $h > 0$ 是带宽参数（bandwidth），控制核函数的缩放程度和平滑水平；
- $\hat{f}_h(x)$ 是点 $x$ 处的密度估计值。

### 核函数

在实际应用中，核函数的选择对估计结果的影响相对较小，而带宽的设置对估计曲线的光滑程度影响较大。

核函数的作用可以理解为：在估计点 $x$ 处，根据样本点 $x_i$ 与 $x$ 之间的距离，赋予不同的权重。距离 $x$ 越近的样本点，其权重越大；距离越远，权重越小。通过对所有样本点的加权平均，得到该点的密度估计。将所有位置的估计值拼接起来，即可得到整体的密度函数曲线。

为了更清楚地理解核函数的加权机制，我们可以对距离进行标准化处理，设：

$$
u_i = \frac{X_i - c}{h}
$$

则以下两式等价：

$$
|u_i| \leq 1 \Longleftrightarrow |X_i - c| \leq h
$$

记 $D_i = |X_i - c|$，表示第 $i$ 个观察值与估计点 $c$ 的距离。核函数的任务就是为每个 $D_i$ 分配权重。

如下图所示，三种典型核函数的权重分配机制具有显著差异：

<img style="width: 600px" src="https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/Fig-NP-kernel-fn01.png">

- Uniform 核：在 $|u| \leq 1$ 范围内赋予所有观察值相同的权重，超出范围的样本点权重为 0 (相当于弃之不用)。对应的密度估计不具有平滑性，常用于教学演示。
- Triangle 核：采用线性下降的加权方式，距离估计点越近权重越大，边界处权重为 0，估计结果具有一定的连续性。
- Epanechnikov 核：采用抛物线型权重函数，在 $u=0$ 处取得最大值，具有最小均方误差（MSE）性质，估计曲线光滑、效率较高。
- Gaussian 核：采用正态分布函数，所有样本点均有非零权重，平滑程度高，适用于大多数实际应用场景。

### 核函数的性质

**常见核函数及其表达式：**

- **Uniform 核函数** $K(u) = \frac{1}{2} \cdot \mathbf{1}\{\left|u\right| \leq 1\}$ （也称为 Rectangular 核函数）

- **Triangle 核函数** $K(u) = (1 - \left|u\right|) \cdot \mathbf{1}\{\left|u\right| \leq 1\}$
- **Epanechnikov 核函数** $K(u) = \frac{3}{4}(1 - u^2) \cdot \mathbf{1}\{\left|u\right| \leq 1\}$
- **Quartic 核函数** $K(u) = \frac{15}{16}(1 - u^2)^2 \cdot \mathbf{1}\{\left|u\right| \leq 1\}$
- **Triweight 核函数** $K(u) = \frac{35}{32}(1 - u^2)^3 \cdot \mathbf{1}\{\left|u\right| \leq 1\}$
- **Gaussian 核函数** $K(u) = \frac{1}{\sqrt{2\pi}} \exp\left(-\frac{u^2}{2}\right)$
- **Cosinus 核函数** $K(u) = \frac{\pi}{4} \cos\left(\frac{\pi}{2} u\right) \cdot \mathbf{1}\{\left|u\right| \leq 1\}$

<img style="width: 600px" src="https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/Fig-NP-kernel-fn02.png">

核函数通常需要满足以下数学性质：

1. 非负性：$K(u) \geq 0$
2. 单位积分：$\int_{-\infty}^{\infty} K(u) \, du = 1$
3. 对称性：$K(u) = K(-u)$
4. 有限的二阶矩：$\int u^2 K(u) \, du < \infty$

实际使用中，还有一些细节需要注意。例如，部分文献或软件将 $\mathbf{1}\{|u| \leq 1\}$ 写为 $\mathbf{1}\{|u| < 1\}$。对于连续变量，两者几乎没有区别；但若数据是离散型的（如整数型变量），则可能影响边界值是否被纳入计算。

核密度估计的构造可以理解为：以每一个样本点为中心放置一个缩放后的核函数，然后在每一个估计位置 $x$ 上，取所有样本点的核值加权平均。因此，它是一种基于样本加权“局部贡献”的整体平滑过程。

总结而言：

- 核函数定义了如何根据样本点与估计点之间的距离分配权重；
- 带宽参数决定了每个样本点的影响范围；
- 合理选择核函数和带宽参数是核密度估计中最关键的步骤；
- 核密度估计为我们提供了一种平滑、灵活且无需模型假设的分布估计方法，广泛应用于经济学、金融学、机器学习等领域的探索性数据分析任务中。
